In [ ]:
import os

def get_preamble():
    # this gets flagged by detect secrets if done in multiline string, no way to flag false positive for specific lines in multiline
    return "REPO_ROOT=/opt/gpudata/steven/label-boosted-RAG-for-RRG\nMIMIC_CXR_DIR=/opt/gpudata/mimic-cxr\nCHEXPERTPLUS_DIR=/opt/gpudata/chexpertplus\nLABEL_DIR=/opt/gpudata/cxr-derived\nBASE_OUTPUT_DIR=/opt/gpudata/labrag\n\nset -e\n"


def get_label_name_from_true_label_path(true_label_csv: str) -> str:
    # assumes some naming conventions of the true labels path like:
    # {dataset}-{section}-labels-{labeler}.csv
    if true_label_csv.endswith("-chexbert.csv"):
        label_name = "chexbert-true"
    elif true_label_csv.endswith("-chexpert.csv"):
        label_name = "chexpert-true"
    else:
        label_name = "unknown"
    return label_name


def get_label_name_from_predicted_label_path(predicted_label_csv: str) -> str:
    # assumes some naming conventions of the predicted labels path like:
    # /[...]/{dataset}-{section}-{model}-classifiers-{labeler}/pred_{threshold}.csv
    # e.g. /[...]/mimic-findings-biovilt-classifiers-chexbert/pred_pr.csv

    filename = os.path.basename(predicted_label_csv)
    full_dir = os.path.dirname(predicted_label_csv)
    direct_parent = os.path.basename(full_dir)

    if "biovilt" in direct_parent:
        emb_model = "biovilt"
    elif "gloria" in direct_parent:
        emb_model = "gloria"
    elif "resnet50" in direct_parent:
        emb_model = "resnet50"
    else:
        emb_model = "unknown"

    if "-chexpert" in direct_parent:
        labeler = "chexpert"
    elif "-chexbert" in direct_parent:
        labeler = "chexbert"
    else:
        labeler = "unknown"

    if "_pr" in filename:
        ths_meth = "pr"
    elif "_roc" in filename:
        ths_meth = "roc"
    else:
        ths_meth = "unknown"

    label_name = f"{emb_model}-{labeler}-{ths_meth}-pred"
    return label_name


def get_exp_str(**kwargs):
    if kwargs["pred_label_csv"] not in ["None", "none", "''", '""']:
        label_type = get_label_name_from_predicted_label_path(kwargs["pred_label_csv"])
    else:
        label_type = get_label_name_from_true_label_path(kwargs["true_label_csv"])

    kwargs["label_type"] = label_type

    kwargs["model_name"] = os.path.basename(kwargs["llm"])

    return """
python $REPO_ROOT/rrg/generate.py \\
--model {llm} \\
--filter_type {filter} \\
--prompt_type {prompt} \\
--section_type {section} \\
--k {k} \\
--batch_size 32 \\
--prompt_yaml $REPO_ROOT/rrg/prompts.yaml \\
--split_csv {split_csv} \\
--metadata_csv {metadata_csv} \\
--true_label_csv {true_label_csv} \\
--predicted_label_csv {pred_label_csv} \\
--report_csv {report_csv} \\
--feature_h5 {emb_h5} \\
--output_dir {output_dir}

python $REPO_ROOT/rrg/eval.py \\
--report_csv {output_dir}/{section}_top-{k}_{label_type}-label_{filter}_{prompt}_{model_name}.csv \\
--output_csv {output_dir}/{section}_top-{k}_{label_type}-label_{filter}_{prompt}_{model_name}_METRICS.csv
""".format(**kwargs)

In [ ]:
DEFAULT_LLM = "mistralai/Mistral-7B-Instruct-v0.3"
DEFAULT_FILTER = "exact"
DEFAULT_PROMPT = "simple"
DEFAULT_LABELS = "chexbert"
DEFAULT_K = 5

for dataset in ["mimic", "chexpertplus"]:
    if dataset == "mimic":
        split_csv = "$MIMIC_CXR_DIR/mimic-cxr-2.0.0-split.csv"
        metadata_csv = "$MIMIC_CXR_DIR/mimic-cxr-2.0.0-metadata.csv"
        report_csv = "$MIMIC_CXR_DIR/mimic_cxr_sectioned.csv"
        emb_model = "biovilt"
        emb_h5 = f"$BASE_OUTPUT_DIR/mimic-cxr-{emb_model}.h5"
    elif dataset == "chexpertplus":
        split_csv = "$CHEXPERTPLUS_DIR/split.csv"
        metadata_csv = "$CHEXPERTPLUS_DIR/metadata.csv"
        report_csv = "$CHEXPERTPLUS_DIR/report.csv"
        emb_model = "gloria"
        emb_h5 = f"$BASE_OUTPUT_DIR/chexpertplus-{emb_model}.h5"
    else:
        raise ValueError(f"Unknown dataset: {dataset}")

    for section in ["findings", "impression"]:
        script_file = f"3-run-generate-eval-{dataset}-{section}.sh"
        true_label_csv = f"$LABEL_DIR/{dataset}-{section}-labels-{DEFAULT_LABELS}.csv"
        pred_label_csv = f"$BASE_OUTPUT_DIR/{dataset}-{section}-{emb_model}-classifiers-{DEFAULT_LABELS}/pred_pr.csv"
        output_dir = f"$BASE_OUTPUT_DIR/exp-{dataset}/exp-{section}"

        base_kwargs = {
            "llm": DEFAULT_LLM,
            "filter": DEFAULT_FILTER,
            "prompt": DEFAULT_PROMPT,
            "k": DEFAULT_K,
            "section": section,
            "split_csv": split_csv,
            "metadata_csv": metadata_csv,
            "report_csv": report_csv,
            "true_label_csv": true_label_csv,
            "pred_label_csv": pred_label_csv,
            "emb_h5": emb_h5,
        }

        cmds = []

        cmds.append("# Core Experiments")
        base_kwargs["output_dir"] = f"{output_dir}/exp-core"
        for filter_type, prompt_type in [("no-filter", "naive"), ("exact", "naive"), ("no-filter", "simple"), ("exact", "simple")]:
            kwargs = base_kwargs.copy()
            kwargs["filter"] = filter_type
            kwargs["prompt"] = prompt_type
            cmds.append(get_exp_str(**kwargs))

        cmds.append("# Filter Experiments")
        base_kwargs["output_dir"] = f"{output_dir}/exp-filter"
        for filter_type in ["no-filter", "exact", "partial"]:
            kwargs = base_kwargs.copy()
            kwargs["filter"] = filter_type
            cmds.append(get_exp_str(**kwargs))

        cmds.append("# Prompt Experiments")
        base_kwargs["output_dir"] = f"{output_dir}/exp-prompt"
        for prompt_type in ["naive", "simple", "verbose", "instruct"]:
            kwargs = base_kwargs.copy()
            kwargs["prompt"] = prompt_type
            cmds.append(get_exp_str(**kwargs))

        cmds.append("# LLM Experiments")
        base_kwargs["output_dir"] = f"{output_dir}/exp-llm"
        for temp in ["mistralai/Mistral-7B-Instruct-v0.3", "mistralai/Mistral-7B-Instruct-v0.1", "BioMistral/BioMistral-7B"]:
            kwargs = base_kwargs.copy()
            kwargs["llm"] = temp
            cmds.append(get_exp_str(**kwargs))

        cmds.append("# Embedding Experiments")
        base_kwargs["output_dir"] = f"{output_dir}/exp-embedding"
        for temp in [emb_model, "resnet50"]:
            kwargs = base_kwargs.copy()
            dset = "mimic-cxr" if dataset == "mimic" else dataset
            kwargs["emb_h5"] = f"$BASE_OUTPUT_DIR/{dset}-{temp}.h5"
            kwargs["pred_label_csv"] = f"$BASE_OUTPUT_DIR/{dataset}-{section}-{temp}-classifiers-chexbert/pred_pr.csv"
            cmds.append(get_exp_str(**kwargs))

        cmds.append("# True Label Experiments")
        base_kwargs["output_dir"] = f"{output_dir}/exp-true-label"
        for temp_true, temp_pred in [
            (f"$LABEL_DIR/{dataset}-{section}-labels-chexbert.csv", f"$BASE_OUTPUT_DIR/{dataset}-{section}-{emb_model}-classifiers-chexbert/pred_pr.csv"),
            (f"$LABEL_DIR/{dataset}-{section}-labels-chexpert.csv", f"$BASE_OUTPUT_DIR/{dataset}-{section}-{emb_model}-classifiers-chexpert/pred_pr.csv"),
            (f"$LABEL_DIR/{dataset}-{section}-labels-chexbert.csv", "None"),
            (f"$LABEL_DIR/{dataset}-{section}-labels-chexpert.csv", "None"),
        ]:
            kwargs = base_kwargs.copy()
            kwargs["true_label_csv"] = temp_true
            kwargs["pred_label_csv"] = temp_pred
            cmds.append(get_exp_str(**kwargs))

        cmds.append("# Top K Experiments")
        base_kwargs["output_dir"] = f"{output_dir}/exp-top-k"
        for temp in [3, 5, 10]:
            kwargs = base_kwargs.copy()
            kwargs["k"] = temp
            cmds.append(get_exp_str(**kwargs))

        with open(script_file, "w") as f:
            f.write(get_preamble())
            for cmd in cmds:
                if cmd.startswith("#"):
                    f.write("\n")
                    f.write("# =================================\n")
                    f.write(cmd + "\n")
                    f.write("# =================================\n")
                else:
                    f.write(cmd)